In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
            .builder \
            .getOrCreate()

21/11/28 16:45:03 WARN Utils: Your hostname, cevat resolves to a loopback address: 127.0.1.1; using 192.168.1.227 instead (on interface wlp4s0)
21/11/28 16:45:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/28 16:45:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
csv_path = "/home/cevat/workspace/data-engineer-project/netflix_titles.csv"
df = spark.read.option("header", "true").csv(csv_path)
df.printSchema()


root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [4]:
## Str to Date format
# to_date adlı fonksiyon ile aşağıdaki spark' ın desteklediği date formatlarına göre veriyi çevirdim. Veri date formatının default verisi olan yyyy-MM-dd formatına çevirilir.
# https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
# to_date' e alternatif olarak to_timestamp fonksiyonuda vardır. O da aynı mantıkla çalışır
from pyspark.sql.functions import *

df = df.select("date_added", to_date(df["date_added"], "MMMM d, yyyy").alias("aa"))
df.show()

+------------------+----------+
|        date_added|        aa|
+------------------+----------+
|September 25, 2021|2021-09-25|
|September 24, 2021|2021-09-24|
|September 24, 2021|2021-09-24|
|September 24, 2021|2021-09-24|
|September 24, 2021|2021-09-24|
|September 24, 2021|2021-09-24|
|September 24, 2021|2021-09-24|
|September 24, 2021|2021-09-24|
|September 24, 2021|2021-09-24|
|September 24, 2021|2021-09-24|
|September 24, 2021|2021-09-24|
|September 23, 2021|2021-09-23|
|September 23, 2021|2021-09-23|
|September 22, 2021|2021-09-22|
|September 22, 2021|2021-09-22|
|September 22, 2021|2021-09-22|
|September 22, 2021|2021-09-22|
|September 22, 2021|2021-09-22|
|September 22, 2021|2021-09-22|
|September 22, 2021|2021-09-22|
+------------------+----------+
only showing top 20 rows



In [10]:
from pyspark.sql.types import LongType, StructType,StructField, StringType, IntegerType, ArrayType, TimestampType, DateType
df = df.withColumn("release_year", df["release_year"].cast(IntegerType()))


from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

func =  udf (lambda x: datetime.strptime(x, '%B %d, %Y').strftime("%Y-%m-%d"), DateType())

df = df.withColumn('test', func(col('date_added')))

df.printSchema()
df.show()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)
 |-- test: date (nullable = true)



PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for item in iterator:
  File "/opt/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 452, in mapper
    result = tuple(f(*[a[o] for o in arg_offsets]

In [16]:
df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|      null|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|      null|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, T